## Application of the Dulmage-Mendelsohn partition to a dynamic chemical looping combustion reactor model
This example constructs a chemical looping combustion reduction reactor using IDAES models, and attempts to fix degrees of freedom to obtain a square and nonsingular model. The Dulmage-Mendelsohn partition is used to debug the case where we unexpectedly have non-zero degrees of freedom as well as the case where we have zero degrees of freedom but a structurally singular model.

### Imports
We will use Pyomo and IDAES, as well as the Pyomo extension "Incidence Analysis." `IncidenceGraphInterface` provides an interface to perform some simple graph algorithms on Pyomo models.

In [ ]:
import pyomo.environ as pyo 
from pyomo.contrib.incidence_analysis import IncidenceGraphInterface

import idaes.core as idaes
from idaes.core.util.model_statistics import degrees_of_freedom

from idaes.gas_solid_contactors.unit_models import MBR as MovingBed
from idaes.gas_solid_contactors.properties.methane_iron_OC_reduction import (
    GasPhaseParameterBlock,
    SolidPhaseParameterBlock,
    HeteroReactionParameterBlock,
)

### Construct and discretize the dynamic model
This will look familiar if you have used IDAES before. Otherwise the important thing is that after this code we are left with a fully discretized (in length and time) dynamic model of the chemical looping combustion (CLC) reactor.

In [ ]:
m = pyo.ConcreteModel()
horizon = 1500.0
fs_config = {
    "dynamic": True,
    "time_units": pyo.units.s,
    "time_set": [0.0, horizon],
}
m.fs = idaes.FlowsheetBlock(default=fs_config)
m.fs.gas_properties = GasPhaseParameterBlock()
m.fs.solid_properties = SolidPhaseParameterBlock()
rxn_config = {
    "solid_property_package": m.fs.solid_properties,
    "gas_property_package": m.fs.gas_properties,
}
m.fs.hetero_reactions = HeteroReactionParameterBlock(default=rxn_config)

nxfe = 5
xfe_list = [1.0*i/nxfe for i in range(nxfe + 1)]
mb_config = {
    "has_holdup": True,
    "finite_elements": nxfe,
    "length_domain_set": xfe_list,
    "transformation_method": "dae.collocation",
    "transformation_scheme": "LAGRANGE-RADAU",
    "pressure_drop_type": "ergun_correlation",
    "gas_phase_config": {
        "property_package": m.fs.gas_properties,
    },      
    "solid_phase_config": {
        "property_package": m.fs.solid_properties,
        "reaction_package": m.fs.hetero_reactions,
    },
}
m.fs.moving_bed = MovingBed(default=mb_config)

time = m.fs.time
t0 = time.first()
disc = pyo.TransformationFactory("dae.finite_difference")
ntfe = 5
disc.apply_to(m, wrt=time, nfe=ntfe, scheme="BACKWARD")

### Check degrees of freedom of model

In [ ]:
print(degrees_of_freedom(m))

### Fix degrees of freedom in the reactor model

In [ ]:
# Fix geometry variables
m.fs.moving_bed.bed_diameter.fix()
m.fs.moving_bed.bed_height.fix()

# Fix dynamic inputs (inlet conditions) at every point in time
m.fs.moving_bed.gas_inlet.flow_mol[:].fix()
m.fs.moving_bed.gas_inlet.pressure[:].fix()
m.fs.moving_bed.gas_inlet.temperature[:].fix()
m.fs.moving_bed.gas_inlet.mole_frac_comp[:, "CO2"].fix()
m.fs.moving_bed.gas_inlet.mole_frac_comp[:, "H2O"].fix()
m.fs.moving_bed.gas_inlet.mole_frac_comp[:, "CH4"].fix()
m.fs.moving_bed.solid_inlet.flow_mass[:].fix()
m.fs.moving_bed.solid_inlet.temperature[:].fix()
m.fs.moving_bed.solid_inlet.particle_porosity[:].fix()
m.fs.moving_bed.solid_inlet.mass_frac_comp[:, "Fe2O3"].fix()
m.fs.moving_bed.solid_inlet.mass_frac_comp[:, "Fe3O4"].fix()
m.fs.moving_bed.solid_inlet.mass_frac_comp[:, "Al2O3"].fix()

# Fix differential variables at the initial time point
m.fs.moving_bed.gas_phase.material_holdup[t0, ...].fix()
m.fs.moving_bed.gas_phase.energy_holdup[t0, ...].fix()
m.fs.moving_bed.solid_phase.material_holdup[t0, ...].fix()
m.fs.moving_bed.solid_phase.energy_holdup[t0, ...].fix();

### Check degrees of freedom again

In [ ]:
print(degrees_of_freedom(m))

We got something wrong and fixed too many degrees of freedom. Instead of guessing, we can use the Dulmage-Mendelsohn partition to tell us which variables are overspecified.

In [ ]:
igraph = IncidenceGraphInterface(m)
var_dmp, con_dmp = igraph.dulmage_mendelsohn()

print("Overconstrained variables:")
for var in var_dmp.overconstrained:
    print("  %s" % var.name)
print("Overconstraining equations:")
for con in con_dmp.overconstrained:
    print("  %s" % con.name)
for con in con_dmp.unmatched:
    print("  %s" % con.name)

### Use our expertise to come up with a fix
We need to decide which variables should have been solved for by eight of these equations. The Dulmage-Mendelsohn partition won't do this for us, in part because the choice of variables to unfix is not unique.

The extra equations seem to be holdup calculation equations at `t0` and `x0` or `xf`, depending on phase. The correct solution is to unfix initial conditions that overlap with boundary conditions (or boundary conditions that overlap with initial conditions).

In [ ]:
x0 = m.fs.moving_bed.length_domain.first()
xf = m.fs.moving_bed.length_domain.last()
m.fs.moving_bed.gas_phase.material_holdup[t0, x0, ...].unfix()
m.fs.moving_bed.gas_phase.energy_holdup[t0, x0, ...].unfix()
m.fs.moving_bed.solid_phase.material_holdup[t0, xf, ...].unfix()
m.fs.moving_bed.solid_phase.energy_holdup[t0, xf, ...].unfix();

### Re-check degrees of freedom

In [ ]:
print(degrees_of_freedom(m))

### Make sure the model is structurally nonsingular

In [ ]:
# Re-construct the interface to make sure unfixed variables
# are included.
igraph = IncidenceGraphInterface(m)
N = len(igraph.constraints)
M = len(igraph.variables)
matching = igraph.maximum_matching()
# If a maximum matching contains all constraints and variables,
# the square model is structurally nonsingular
print(N, M, len(matching))

### What if we didn't use Dulmage-Mendelsohn and chose a slightly different fix
Such as unfixing solid phase differential variables at `t0` and `x0` instead of `xf`

In [ ]:
# Fix solid phase initial conditions at x0
m.fs.moving_bed.solid_phase.material_holdup[t0, x0, ...].unfix()
m.fs.moving_bed.solid_phase.energy_holdup[t0, x0, ...].unfix()
# And unfix them at xf
m.fs.moving_bed.solid_phase.material_holdup[t0, xf, ...].fix()
m.fs.moving_bed.solid_phase.energy_holdup[t0, xf, ...].fix();

### Re-check degrees of freedom

In [ ]:
print(degrees_of_freedom(m))

Looks good!

### Check for structural nonsingularity

In [ ]:
igraph = IncidenceGraphInterface(m)
N = len(igraph.constraints)
M = len(igraph.variables)
matching = igraph.maximum_matching()
print(N, M, len(matching))

But a maximum matching doesn't include all the constraints and variables, so the model is structurally singular.

In [ ]:
var_dmp, con_dmp = igraph.dulmage_mendelsohn()
print("Underconstrained variables:")
for var in var_dmp.unmatched:
    print("  %s" % var.name)
for var in var_dmp.underconstrained:
    print("  %s" % var.name)
print("Overconstraining equations:")
for con in con_dmp.unmatched:
    print("  %s" % con.name)
for con in con_dmp.overconstrained:
    print("  %s" % con.name)

We have unmatched variables at `x0` and unmatched constraints at `xf`

So we really did need to unfix solid phase initial conditions only at `xf`

In [ ]:
m.fs.moving_bed.solid_phase.material_holdup[t0, x0, ...].fix()
m.fs.moving_bed.solid_phase.energy_holdup[t0, x0, ...].fix()
m.fs.moving_bed.solid_phase.material_holdup[t0, xf, ...].unfix()
m.fs.moving_bed.solid_phase.energy_holdup[t0, xf, ...].unfix()

print(degrees_of_freedom(m))
igraph = IncidenceGraphInterface(m)
N = len(igraph.constraints)
M = len(igraph.variables)
matching = igraph.maximum_matching()
print(N, M, len(matching))

### Take-aways from this example:
- Just because a model has zero degrees of freedom doesn't mean its specification is "correct"
- The Dulmage-Mendelsohn partition can help when you expect to have zero degrees of freedom, but don't
- Always check whether your (square) model has a perfect matching of equations and variables
- If it doesn't, you can use the Dulmage-Mendelsohn partition to help find out why